# Reranking file

## Libray imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # Go up one directory to see src/
from datetime import datetime
import pandas as pd

### Import files

In [2]:
from src.reranker import rerank_routes
from src.route_finder import find_routes
from src.crowd_predictor import load_model, predict_crowdedness
from src.summarizer import generate_trip_summary
from src.user_input_planner import plan_trip

## Graph simulation of routes

In [3]:
graph = {
    "15L": ["A","B","C","D","E"],
    "0": ["F","C","G"],
    "1": ["H","I","J","E"],
}

## Find routes

In [4]:
candidates = find_routes("B","E", graph)

## Mock crowd, delay, and safety scores

In [5]:
for c in candidates:
    c["walk_time"] = 5
    c["delay_minutes"] = 4
    c["crowdedness_score"] = 30 # mock
    c["safety_score"] = 70 # mock

## Turn into Data frame for reranking

In [6]:
df = pd.DataFrame(candidates)

## Score and sort the results

In [7]:
reranked = rerank_routes(df)

## Display

In [8]:
display(reranked)

,route_id,origin,destination,stops,bus_time,walk_time,delay_minutes,crowdedness_score,safety_score,total_score
0,15L,B,E,"[B, C, D, E]",12,5,4,30,70,18.5


In [9]:
model = load_model("models/crowd_model.pkl")

## Assume 'df' contains route_id, hour_of_day, day_of_week

In [10]:
df["hour_of_day"] = 17
df["day_of_week"] = "Monday"

## Predict crowdedness (returns 0==Low, 1==Medium, 2==High)

In [11]:
df["crowdedness_score"] = predict_crowdedness(model, df)

### Optional: Convert back to readable labels

In [12]:
label_map = {0: "Low", 1: "Medium", 2: "High"}
df["crowdedness_label"] = df["crowdedness_score"].map(label_map)

## Display top candidates

In [13]:
display(df[["route_id", "origin", "destination", "crowdedness_score", "crowdedness_label"]])

,route_id,origin,destination,crowdedness_score,crowdedness_label
0,15L,B,E,1,Medium


## Mock trip planner

In [14]:
trip = {
    "walk_time": 5,
    "route_id": "15L",
    "stop_name_start": "Blake & 14th",
    "stop_name_end": "Decatur Station",
    "departure_time": "7:14 PM",
    "arrival_time": "7:38 PM",
    "destination": "840 Irving Street"
}

## Summarize the trip

In [15]:
summary = generate_trip_summary(trip)
print(summary)

Walk 5 minutes to Blake & 14th. Take the 15L bus at 7:14 PM. Get off at Decatur Station. You’ll arrive at 840 Irving Street by 7:38 PM.


## User defined trip details

In [16]:
user_start = "B"
user_end = "E"
user_time = datetime.strptime("2025-04-16 17:00", "%Y-%m-%d %H:%M")

## Generate reranked trips

In [17]:
results, msg = plan_trip(user_start, user_end, user_time, graph, weather="Rainy")
print(msg)


✅ Trip plan ready!


## Show top trip options with summaries

In [18]:
results[["route_id", "crowdedness_label", "total_score", "trip_summary"]].head()

,route_id,crowdedness_label,total_score,trip_summary
0,15L,🟡 Medium,60.3,Walk 5 minutes to B. Take the 15L bus at 5:00 ...
